# Coursework 2 
### Perform k-nearest neighbour (KNN) retrieval experiments according to standard practices in pattern recognition. Use retrieval error (ie @rank1, @rank10) as the performance metric to evaluate different methods. Your baseline approach is KNN on provided features. Use distance metric learning methods to improve a baseline performance.

#### So, firstly, we need to create a baseline, which is KNN on provided features. 

## So there is a training, validation, testing

### loading the data cuhk03 new protocol config labeled 

In [4]:
from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()

In [5]:
print(train_idxs)

[    1     2     3 ... 14094 14095 14096]


In [6]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [9]:
cam_Ids = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
cam_Ids

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [10]:
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
filelists

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [11]:
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [12]:
labelss = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
labelss

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [13]:
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [14]:
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [45]:
print(cam_Ids)

[1 1 1 ... 2 2 2]


## Load features

In [17]:
import json
import numpy as np
import matplotlib.pyplot as plt

with open("PR_data/feature_data.json", "r") as file:
    features = json.load(file)
    
data_features = np.asarray(features)

print('Data shape: {}'.format(data_features.shape))

Data shape: (14096, 2048)


In [19]:
#Load labels
labelss = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()

#Load camId
cam_Ids = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

#Load indexes
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()

#Load training indexes
print("training indexes : {}".format(train_idxs.shape))
print("query indexes : {}".format( query_idxs.shape))
print("gallery indexes : {}".format( gallery_idxs.shape))

training indexes : (7368,)
query indexes : (1400,)
gallery indexes : (5328,)


In [34]:
train_idxs = train_idxs - 1
query_idxs = query_idxs - 1
gallery_idxs = gallery_idxs - 1

In [42]:
train_f = []


# divide the features data into training 
for i in range (len(train_idxs)):
    train_f.append(data_features[train_idxs[i]])
    
np.asarray(train_f).shape    


(7368, 2048)

In [43]:
#features data into query
query_f = []
for i in range (len(query_idxs)):
    query_f.append(data_features[query_idxs[i]])
    
np.asarray(query_f).shape  

(1400, 2048)

In [44]:
#features data into gallery
gallery_f = []
for i in range (len(gallery_idxs)):
    gallery_f.append(data_features[gallery_idxs[i]])
    
np.asarray(gallery_f).shape  

(5328, 2048)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 1) 
classifier.fit(gallery_f[1], query_f)  #fitting data
face_predict = classifier.predict(face_test_pca.real)

